In [8]:
from pymongo import MongoClient
import pandas as pd

In [80]:
def connect_database(database_name : str):
    client = MongoClient()
    return client[database_name]

def get_collection(connection, collection_name : str) :
    return connection[collection_name]

In [101]:
def read_and_create_dataframe(collection):
    df = pd.DataFrame()
    for game in collection.find() :
        df = pd.concat([df,pd.json_normalize(game)])
    df = df.explode('participants').reset_index(drop=True)
    df_participants = pd.json_normalize(df['participants'])
    df = pd.concat([df.drop(columns='participants'),df_participants],axis = 1)
    return df

In [ ]:
connect = connect_database('lol_match_database')
scrim_matches = get_collection(connect,"scrim_matches")
data_scrim_matches = read_and_create_dataframe(scrim_matches)
data_scrim_matches